In [1]:
from typing import Optional, List
from itertools import product
from importlib import reload

from numpy import sqrt
import pandas as pd
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)

import plotly.express as px
import plotly.io as pio

from data import build_dataset_with_composites, get_factor_master, build_factor_data2
from stats import total_return
from util import format_date

from market_feedback import  draw_market_feedback_scatter #px_scatter,



In [2]:
# factor_data.to_pickle('factor_data.pkl')

In [ ]:
halflifes = [21, 63, 126, 252]
factor_data = build_factor_data2(halflifes, factor_set='read')

In [ ]:
return_start = '2024-12-31'
return_end = '2025-01-28'
return_title = f'Month-to-Date Return (std, as of {format_date(return_end)})'

# return_start = '2025-01-21'
# return_end = '2025-01-28'
# return_title = f'5-day Return (std, {format_date(return_start)} - {format_date(return_end)})'

vol_type = 63
corr_type = 63
corr_asset = 'IEF'

fig = draw_market_feedback_scatter(factor_data, return_start, return_end, vol_type, corr_type, corr_asset, return_title)
fig.show(renderer='png')
# fig.write_html('fed_feedback.html')
# fig.write_image('fed_feedback.png')


# Loop through market feedback

In [51]:
def get_intervals(ds: xr.Dataset) -> pd.DataFrame:
    # TODO: Accept list of intervals, say [5, 21, 63]
    dates = factor_data.indexes['date']
    date_latest = dates.max()
    year_start = pd.Timestamp(date_latest.year, 1, 1)
    month_start = pd.Timestamp(date_latest.year, date_latest.month, 1)
    ytd_start = dates[dates < year_start].max()  # prior_year_end
    mtd_start = dates[dates < month_start].max() # prior_month_end
    wow_start = dates[-5-1]                        # week_over_week
    
    return (pd.DataFrame(
                columns = ['name', 'start_date', 'end_date',   'chart_title'],
                data =   [['ytd',   ytd_start,    date_latest, 'Year-to-Date Return (std)'],
                          ['mtd',   mtd_start,    date_latest, 'Month-to-Date Return (std)'],
                          ['5d',    wow_start,    date_latest, '5-Day Return (std)'],
                          ])
            .set_index('name'))


In [ ]:
def draw_market_feedback_scatter_set(factor_data, corr_asset_list, vol_type, corr_type):
    # TODO: Add output path
    # TODO: Add dates (and source) to chart
    intervals = get_intervals(factor_data)
    interval_list = intervals.index.to_list()

    for corr_asset, interval in product(corr_asset_list, interval_list):
        i = intervals.loc[interval]
        fig = draw_market_feedback_scatter(factor_data, i['start_date'], i['end_date'], vol_type, corr_type, corr_asset, i['chart_title'])
        fig.show(renderer='png')
        fig.write_image(f'feedback_{corr_asset}_{interval}.png')
        # fig.write_html(f'feedback_{corr_asset}_{interval}.html')

corr_asset_list = ['SPY', 'IEF', 'GLD']
vol_type = 63
corr_type = 63
draw_market_feedback_scatter_set(factor_data, corr_asset_list, vol_type, corr_type)

# Correlation Matrix

In [7]:
def sort_assets(corr: pd.DataFrame, sorting_factor: str, factor_master: pd.DataFrame) -> pd.Index:
    df = (corr[[sorting_factor]]
          .join(factor_master[['asset_class', 'hyper_factor']])
          .assign(is_theme=lambda df: df['asset_class'] == 'Theme')
          .assign(is_sorting_factor=lambda df: df.index == sorting_factor)
          )
    sorted_index = df.sort_values(by=['is_sorting_factor', 'hyper_factor', 'is_theme', sorting_factor], 
                                  ascending=[False, False, True, False], key=abs).index
    return sorted_index


In [ ]:
date_latest = factor_data.date.max().values
corr = factor_data.corr.sel(corr_type=corr_type, date=date_latest).to_pandas()

corr



def format_corr_matrix(corr: pd.DataFrame): # -> pd.io.formats.style.Styler:
    """
    Format the correlation matrix by adding asset class information and sorting.
    
    Parameters
    ----------
    corr : pd.DataFrame
        The correlation matrix with assets as both rows and columns.
    
    Returns
    -------
    pd.io.formats.style.Styler
        The formatted correlation matrix as a pandas Styler object.
    """
    # Sort the correlation matrix by asset class
    corr = corr
    
    # Style the correlation matrix
    styled_corr = corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1).format(precision=2)
    
    return styled_corr

# Apply the function to format the correlation matrix
formatted_corr = format_corr_matrix(corr) #, factor_master['asset_class'])
formatted_corr



In [ ]:
factor_master = get_factor_master()

date_t0 = factor_data.date.max().values
date_t1  =factor_data.date[-63].values

corr_t0 = factor_data.corr.sel(corr_type=corr_type, date=date_t0).to_pandas()
corr_t1 = factor_data.corr.sel(corr_type=corr_type, date=date_t1).to_pandas()
corr_diff = corr_t0 - corr_t1
sorted_index = sort_assets(corr_t0, '2s10s', factor_master)
sorted_index_chg = sort_assets(corr_diff, '2s10s', factor_master)

format_corr_matrix(corr.loc[sorted_index, sorted_index])
format_corr_matrix(corr_diff.loc[sorted_index_chg, sorted_index_chg])

# Draw correlation time series

In [ ]:
factor_data

In [ ]:
asset1 = 'SPY'
asset2 = 'IEF'
start_date = '2020'
start_date = None


df1  = factor_data.sel(factor_name=asset1, factor_name_1=asset2, date=slice('2024', None)).corr.to_pandas().T.stack()

from chart import draw_correlation, draw_volatility

fig1 = draw_correlation(factor_data.corr.sel(date=slice(start_date, None)), asset1, asset2, [21, 63, 126, 252])
fig1.show(renderer='png')

fig2 = draw_volatility(factor_data.vol.sel(date=slice(start_date, None)), asset1, [21, 63, 126, 252])
fig3 = draw_volatility(factor_data.vol.sel(date=slice(start_date, None)), asset2, [21, 63, 126, 252])

fig2.show()
fig3.show()

fig1.show(renderer='png')
# fig1.write_html('corr_ts.html')
# fig1.write_image('corr_ts.png')


In [ ]:
start_date = '2015'
start_date = None

corr_pairs = [('SPY', 'IEF'),
            #   ('SHY', 'IEF'), 
            #   ('2s10s', 'SPY'), 
            #   ('2s10s', 'IEF'), 
            #   ('2s10s', 'HYGH'), 
            #   ('2s10s', 'HYG'), 
            #   ('2s10s', 'LQD'), 
              ('2s10s', 'LQDH'), 
            #   ('2s10s', 'VMBS'), 
            #   ('2s10s', 'AGNC')
              ]

for asset1, asset2 in corr_pairs:
    fig = draw_correlation(factor_data.corr.sel(date=slice(start_date, None)), asset1, asset2, [21, 63, 126, 252])
    fig.show(renderer='png')
    # fig.write_html(f'corr_{asset1}_{asset2}.html')
    # fig.write_image(f'corr_{asset1}_{asset2}.png')

In [12]:
fig1.write_html('corr_ts.html')
fig1.write_image('corr_ts.png')

fig.write_html('fed_feedback.html')
fig.write_image('fed_feedback.png')

In [ ]:
# factor_data.ret.sel(asset=[asset1, asset2]).plot(hue='asset')
factor_data.ret.sel(asset='IWM').plot()
factor_data.ret.sel(asset='IEF').plot()
factor_data.ret.sel(asset='SHY').plot()

factor_data.ret.sel(asset='2s10s').plot()